# Assignment 4

Approximate time needed: 6-7 hours
    
## DLT and Epipolar Lines

The goal of this assignment is to aid your understanding of the Direct Linear Transform as well as Epipolar geometry.

For the first part of the assignment you will find correspondences between 2D-3D points and estimate the P Matrix. You will then be required to estimate Camera Parameters from this P matrix.

The second part of this assignment will require you to construct epipolar lines on two corresponding images.

The third part of this assignment will require you to use camera intrinsics to estimate the bounding box of a car.

---
### 4.1 Generating Correspondences
The first step to perform DLT is to generate correspondences. The cell below opens a new window. Clicking anywhere on the image should give you the pixel location of the image. Once you're done clicking, close the image window. The cell after displays the points you have clicked.

In [1]:
%matplotlib tk
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
import numpy as np


# image_points = []
# fig = plt.figure(figsize=(20,30))

# img = mpimg.imread('q1/dlt1.jpeg')

# def onclick(event):
#     ix, iy = event.xdata, event.ydata
#     image_points.append([ix, iy])

# cid = fig.canvas.mpl_connect('button_press_event', onclick)

# imgplot = plt.imshow(img)
# plt.show()

In [2]:
# %matplotlib inline
# N = len(image_points)
# image_points = np.array(image_points)
# fig = plt.figure(figsize=(10,15))

# img=mpimg.imread('q1/dlt1.jpeg')
# imgplot = plt.imshow(img)

# colors = np.random.rand(N)
# area = (15 * np.ones(N))**2 

# print(image_points)
# plt.scatter(image_points[:,0], image_points[:,1], c=colors, s=area)
# plt.show()

In [3]:
x = [[321.94620385, 218.83386385],
 [ 95.09196666, 336.92237089],
 [346.80694218, 496.96337384],
 [517.72451815, 571.54558881],
 [127.72168571, 587.08355026],
 [342.14555374, 739.35557249]]
x = np.array(x)
print(x)

[[321.94620385 218.83386385]
 [ 95.09196666 336.92237089]
 [346.80694218 496.96337384]
 [517.72451815 571.54558881]
 [127.72168571 587.08355026]
 [342.14555374 739.35557249]]


After generating pixel locations, you have to generate the corresponding world points. You have the freedom to chose which point you want as origin. You can consider each side as 4 units.

Ensure that the ordering of your points matches the ordering of image points. Keep track of the sequence of corners clicked.

In [4]:
# x = image_points

# the red   - table edge is X axis
# the green - table edge is Y axis
# the red   - green edge is Z axis

X = []
X.append([4, 4, 4])
X.append([0, 4, 4])
X.append([0, 0, 4])
X.append([4, 0, 0])
X.append([0, 4, 0])
X.append([0, 0, 0])
X = np.array(X)

print("x: ", x.shape)
print("X: ", X.shape)

x:  (6, 2)
X:  (6, 3)


Now that we have the correspondences, we can estimate the P matrix. Implement the DLT function that returns the P matrix

Quick recap: x_i = P * X_i where (x_i, X_i) is the ith correspondence, and P is a 3x4 dimensional matrix. P is further decomposed as P = K * [ R | T].


In [5]:
# Converting this points to homogenous co-ordinates
bb = np.hstack((x, np.ones((x.shape[0], 1))))
bb1 = np.hstack((X, np.ones((X.shape[0], 1))))

In [6]:
def M(measured, control):
    n = measured.shape[0] 
    npz = np.zeros(n)
    a_x = np.c_[-1*control[:, 0], -1*control[:, 1], -1*control[:, 2], -1*np.ones(n), npz, npz, npz, npz, np.multiply(measured[:, 0], control[:, 0]), np.multiply(measured[:, 0], control[:, 1]), np.multiply(measured[:, 0], control[:, 2]), measured[:, 0]]
    a_y = np.c_[npz, npz, npz, npz, -1*control[:, 0], -1*control[:, 1], -1*control[:, 2], -1*np.ones(n), np.multiply(measured[:, 1], control[:, 0]), np.multiply(measured[:, 1], control[:, 1]), np.multiply(measured[:, 1], control[:, 2]), measured[:, 1]]
    out = np.empty((2 * n, a_x.shape[1]))
    out[::2, :] = a_x
    out[1::2, :] = a_y
    return out

def DLT(x,X):
    m = M(x, X)
    u, d, v = np.linalg.svd(m)
    P = v[-1, :]
    P = P.reshape((3, 4))
    return P

P = DLT(bb, bb1)
print(" P matrix:\n", P)


 P matrix:
 [[-7.95512149e-02  6.23843017e-02  1.51278478e-02 -4.15648968e-01]
 [ 2.17822991e-02  3.34676519e-02  9.72071888e-02 -8.97631115e-01]
 [-5.09921039e-05 -2.18598329e-05  4.74009832e-05 -1.21390947e-03]]


### Theory

- Explain how DLT is implemented and show why it works (proof)
- When does DLT fail?

### Solutions

- pass
- pass

## Analysing P

The next part of this question is to analyse how good our estimate is. Report the reprojection error of your P matrix

In [7]:
#Get reprojected points
def get_points(P, X):
    homogenous_loc = np.dot(P, X.T)
    for i in range(len(homogenous_loc)-1):
        for j in range(len(homogenous_loc[i])):
            homogenous_loc[i][j] = homogenous_loc[i][j]/homogenous_loc[2][j]
    return homogenous_loc[:2]

#Get reprojection error
def get_err(x, X, P):
    img = get_points(P, X)
    err = np.linalg.norm(x - img.T, axis=1)
    return np.mean(err ** 2)

print(' Reprojection Error:', get_err(x, bb1, P))

 Reprojection Error: 0.0605138016438434


Given the P matrix, we can estimate K, R and T. Write a function that returns K,R,T given P.
Note: 
P = K * [R | T] where K is a 3x3 matrix and [R | T], P are 3x4 matrices

In [8]:
def ret_RKT(P):
    
    # Calculated T using the matrix structure and R and K using QR decomposition
    H_inf = P[:, :3]
    h = P[:, -1]
    
    T = -np.dot(np.linalg.inv(H_inf), h)
    R_tr, K_inv = np.linalg.qr(np.linalg.inv(H_inf))
    K = np.linalg.inv(K_inv)
    # Haven't done the camera constant correction
    
    return R_tr.T, K/K[-1, -1], T

R, K, T = ret_RKT(P)

print(' K:\n', K)
print('\n R:\n', R)
print('\n T:\n', T)

 K:
 [[ 1.24586561e+03 -7.65291344e+00  6.40359097e+02]
 [-0.00000000e+00 -1.34323630e+03  5.19332513e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]

 R:
 [[-0.51887886  0.83736014 -0.17202538]
 [-0.49239832 -0.45726228 -0.74057754]
 [-0.69879083 -0.29956502  0.64957846]]

 T:
 [-8.2092691  -7.0821848  13.51208672]


#### Verifying the R, K, T Values

Use `np.isclose` to verify the matrix

In [9]:
i3 = np.eye(3)
rt = np.vstack((i3, -T)).T

ans = K@R@rt

print(' P:\n', P/P[-1, -1])
print('\n K * R * [I | -T]:\n', ans/ans[-1, -1])

np.isclose(P/P[-1, -1], ans/ans[-1, -1])

 P:
 [[ 6.55330702e+01 -5.13912306e+01 -1.24620889e+01  3.42405242e+02]
 [-1.79439238e+01 -2.75701381e+01 -8.00777906e+01  7.39454740e+02]
 [ 4.20065128e-02  1.80077949e-02 -3.90482027e-02  1.00000000e+00]]

 K * R * [I | -T]:
 [[ 6.55330702e+01 -5.13912306e+01 -1.24620889e+01  3.42405242e+02]
 [-1.79439238e+01 -2.75701381e+01 -8.00777906e+01  7.39454740e+02]
 [ 4.20065128e-02  1.80077949e-02 -3.90482027e-02  1.00000000e+00]]


array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

**Reproject the world points with P matrix you have obtained via DLT and visualize**

In [10]:
img_pts = get_points(P, bb1)   # 6x2
img_pts = img_pts.T
N = img_pts.shape[0]   # 6 number of points
fig = plt.figure(figsize=(10,15))
plt.clf()
img=mpimg.imread('q1/dlt1.jpeg')
imgplot = plt.imshow(img)
plt.scatter(img_pts[:, 0], img_pts[:, 1], s=200)
plt.title('Using Estimated P')
plt.show()

### Zhangs method

For this task, use the inbuilt Opencv function to estimate the K matrix of a camera. Use the checkerboard images 5456-5470 in `q1/zhangs` folder for this task. Familiarize yourself with the different parameters for this method.

**Reference**: https://docs.opencv.org/3.4/dc/dbb/tutorial_py_calibration.html

In [11]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [21]:
objectlist = np.zeros((48,3), np.float32)
objectlist[:,:2] = np.mgrid[0:6,0:8].T.reshape(-1,2)
li = sorted(os.listdir('q1/Zhangs/'))
objpoints = [] 
imgpoints = [] 

# plt.figure(figsize=(16,8))

for i in range(len(li)):   
    print("Opening ", 'q1/Zhangs/' + li[i])
    img = cv2.imread('q1/Zhangs/' + li[i])
    outp = cv2.findChessboardCorners(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY), (6, 8),None)

    if outp[0]:
        objpoints.append(objectlist)
        corners2 = cv2.cornerSubPix(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY),outp[1],(11,11),(-1,-1),(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        imgpoints.append(corners2)
        img = cv2.drawChessboardCorners(img, (6,8), corners2,outp[0])
#         plt.imshow(img)
        
K = cv2.calibrateCamera(objpoints, imgpoints, cv2.cvtColor(img,cv2.COLOR_BGR2GRAY).shape[::-1],None,None)[1]
# plt.show()
print('\nK:', K)

Opening  q1/Zhangs/IMG_5456.JPG
Opening  q1/Zhangs/IMG_5457.JPG
Opening  q1/Zhangs/IMG_5458.JPG
Opening  q1/Zhangs/IMG_5459.JPG
Opening  q1/Zhangs/IMG_5460.JPG
Opening  q1/Zhangs/IMG_5461.JPG
Opening  q1/Zhangs/IMG_5462.JPG
Opening  q1/Zhangs/IMG_5463.JPG
Opening  q1/Zhangs/IMG_5464.JPG
Opening  q1/Zhangs/IMG_5465.JPG
Opening  q1/Zhangs/IMG_5466.JPG
Opening  q1/Zhangs/IMG_5467.JPG
Opening  q1/Zhangs/IMG_5468.JPG
Opening  q1/Zhangs/IMG_5469.JPG
Opening  q1/Zhangs/IMG_5470.JPG

K: [[1.36415093e+04 0.00000000e+00 3.31635835e+03]
 [0.00000000e+00 1.36632517e+04 1.50037365e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


---
### 4.2 Epipolar lines
#### Task 1

For this task, you have been given two images of the same scene taken from different view-points. You should first estimate the fundamental matrix from these two images.

<img src="./q2/img1.jpg" alt="image 1" width="400"/>
<img src="./q2/img2.jpg" alt="image 2" width="400"/>


Recall that given a point in one image, it's corresponding location in the other image can be found to along a line viz. the epipolar line. The task given to you is to draw the epipolar lines in the second image for each given point in the first image. You have to repeat this for the other image as well. Draw epipolar lines on the first image for the corresponding points in the second image.

The convention used for F is $x'^{T}Fx$ where $x'$ is the location of the point in the second image. For this question you will need to compute the F matrix on your own without using inbuilt functions. 

**Provide a clear explanation on your approach for both this task and the next**

#### Task 2
Report the location of the epipoles. Mathematically find the epipoles and verify that the lines intersect at the epipole.

____
### 4.3 Drawing a bounding box around a car and localizing it in the camera frame
You’ve been provided with an image, also taken from a self-driving car, that shows another car in front. The camera has been placed on top of the car, 1.65 m from the ground, and assume the image plane is perfectly perpendicular to the ground. K is provided to you. Your task is to draw a 3D-bounding box around the car in front as shown. Your approach should be to place eight points in the 3D world such that they surround all the corners of the car, then project them onto the image, and connect the projected image points using lines. You might have to apply a small 5° rotation about the vertical axis to align the box perfectly. Rough dimensions of the car - h: 1.38 m, w: 1.51, l: 4.10. (Hint: Fix a point on the ground as your world origin.). Also estimate the approximate translation vector to the mid-point of the two rear wheels of the car in the camera frame.

![Kitti car](./q3/image.png)

In [22]:
!ls q3
!cat q3/K.txt

image.png*  K.txt*
K = [7.2153e+02,0,6.0955e+02;0,7.2153e+02,1.7285e+02;0,0,1]


In [101]:
def transform(objpoints):
    
    
    
    translation = [2.81492622, 1.65, 9.00888763]
    translation = np.array(translation)
    
    for i in range(len(objpoints)):
        objpoints[i] = objpoints[i] + translation
        
    return objpoints

# [[2.81492622]
#  [1.65      ]
#  [9.00888763]]

height = 1.65
h = 1.38
w = 1.51
l = 4.10

K = [[7.2153e+02,0,6.0955e+02], [0,7.2153e+02,1.7285e+02], [0,0,1]]
K = np.array(K)

objpoints = [[0, height-0, 0],
             [0, height-0, l],
             [0, height-h, 0],
             [0, height-h, l],
             [w, height-0, 0],
             [w, height-0, l],
             [w, height-h, 0],
             [w, height-h, l],
            ]

objpoints = np.array(objpoints)
objpoints = transform(objpoints)

projected = (K@objpoints.T).T

for i in range(len(projected)):
    projected[i] = projected[i]/projected[i][-1]
        
b = np.delete(projected, -1, axis=1)


fig = plt.figure(figsize=(20, 20))
plt.clf()
img=mpimg.imread('q3/image.png')
imgplot = plt.imshow(img)
plt.scatter(b[:, 0], b[:, 1], s=200)
plt.title('Plot with keypoints')
plt.show()

In [94]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
x = [[835], [305], [1]]
K = np.mat(([7.2153e+02,0,6.0955e+02],[0,7.2153e+02,1.7285e+02],[0,0,1]))
B = np.dot(np.linalg.inv(K),x)

h = 1.65
groundNormal = [[0,1,0]]

WorldPoint = h*B/np.dot(groundNormal,B)

print(WorldPoint)

d2point = np.matmul(K,WorldPoint)

carHeight = 1.38
carWidth = 1.51
carLength = 4.10


#5 degree rotation along y clockwise

Rot = np.mat(([np.cos(5*np.pi/180),0,-np.sin(5*np.pi/180)],[0,1,0],[ np.sin(5*np.pi/180),0,np.cos(5*np.pi/180)]))
 
# KITTI dataset y is downwards x is right and z is forward (into the image)
WorldPoint2 = np.matmul(Rot,[[0],[0],[carLength]]) + WorldPoint
WorldPoint3 = np.matmul(Rot,[[carWidth],[0],[0]]) + WorldPoint
WorldPoint4 = np.matmul(Rot,[[0],[-carHeight],[0]]) + WorldPoint
WorldPoint5 = np.matmul(Rot,[[carWidth],[-carHeight],[0]]) + WorldPoint
WorldPoint6 = np.matmul(Rot,[[0],[-carHeight],[carLength]]) + WorldPoint
WorldPoint7 = np.matmul(Rot,[[carWidth],[0],[carLength]]) + WorldPoint
WorldPoint8 = np.matmul(Rot,[[carWidth],[-carHeight],[carLength]]) + WorldPoint

WorldPoints = np.array((WorldPoint,WorldPoint2,WorldPoint3,WorldPoint4,WorldPoint5,WorldPoint6,WorldPoint7,WorldPoint8))

# print(K)
# print(WorldPoints[0])
# print(np.matmul(K,WorldPoint + WorldPoint2 ))
# print(np.matmul(K,WorldPoint + WorldPoint3 ))
# print(K[2][:])

pic = plt.imread("q3/image.png")
plt.imshow(pic)
 
edges = np.array([1, 3, 5, 4, 1, 2, 6, 8, 7, 3, 5, 8, 6, 4, 6, 2, 7]) 
for i in range(8):
    d2point = np.matmul(K,WorldPoints[i])
    d2point = d2point/d2point[2]
    plt.plot(d2point[0],d2point[1],'bo')

# print(K)    
linemat = np.mat(((),(),()))
for i in range(edges.size):
    tmpmat = np.matmul(np.mat(K),np.mat(WorldPoints[edges[i] - 1]))
    tmpmat = tmpmat/tmpmat[2]
    linemat = np.hstack((linemat,tmpmat))

# print(linemat[0])    

plt.plot(np.transpose(linemat[0]),np.transpose(linemat[1]),linewidth="2",c='r')    

plt.show()

[[2.81492622]
 [1.65      ]
 [9.00888763]]
